# Scaled dot product attention

## I. Naive implementation的memory Analysis

### 1. math

- Forward：
$$\begin{align}
S & = \frac{QK^T}{\sqrt{d}} \\
P & = softmax(S)=\frac{exp(S-max_{row}(S))}{\sum_{row} exp(S-max_{row}(S))} \\
A & = PV
\end{align}$$

- Backward：
$$\begin{align}
dV & = P^TdA \\
dP & = dAV^T \\
dS & = P*(dP - sum_{row}(P*dP)) = P*(dP - sum_{row}(A*dA))\\
dQ & = dSK/\sqrt{d} \\
dK & = dS^TQ/\sqrt{d} \\
\end{align}$$

### 2. memory analysis

<img src='pics/attention_memory_analysis.png' width='100%'>

#### (1) Forward

- 计算过程
$$\begin{align}
S & = QK^T               \\
S & = S/\sqrt{d}         \\
Softmax\ call: temp & = max_{row}(S)\\
S & = S-temp              \\
S & = exp(S)     \\
temp & = sum(S)    \\
Softmax\ output: P & = S/temp    \\
A & = PV    \\
\end{align}$$

- 主要的memory space cost是shape为(B, H, T, T)的score相关矩阵，包括下面几个需求。最大空间需求是4倍(B, H, T, T)大小的空间。<font color=red>softmax是memory space的瓶颈。</font>
  1. bmm结果S
  2. div结果S
  3. S作为softmax的input占有space的同时，在softmax计算过程中有3个运算需要空间：substract, exp, div。因此softmax计算过程中最多可能需要4倍(B, H, T, T)大小的空间
  4. P会saved for Backward

- 对应DRAM的load/store包括6次读，5次写。其中大部分也发生在softmax的计算中。
  - (1)式中计算的S存入DRAM
  - (2)式读S，并将scaledS存入DRAM
  - (3)式读scaledS
  - (4)式读scaledS，并将stableS存入DRAM
  - (5)式读stableS，将expS写入DRAM
  - (6)式读expS
  - (7)式读expS，计算P并存入DRAM

#### (2) Backward

- pytorch按照forward中函数定义方式执行的实际backward过程：
  - 主要的memory space cost同样是shape为(B, H, T, T)的score相关矩阵，包括：P, dP, dS, 以及需要的temprary variable。最多需要同时容纳6个该大小的矩阵。
    - <font color=brown>[pickle文件详见cs336 Assignment2 cs336_system中attention文件夹]</font>
    - 内存耗用量最大的环节发生在对$P=expS/sum(expS)$这一步，计算dexpS一共分配了5次(B, H, T, T)形状的空间。

$$\begin{align}
bmmBackward: & &=> dV = P^TdA, dP = dAV^T \\
in\ softmax:\\
divBackward: & P=expS/sum(expS) &=> dexpS \\
expBakkward: & expS=exp(stableS) &=> dstableS \\
subBackward: & stableS=maskedS-max\_maskedS &=> dmaskedS\_part1, dmax\_maskedS \\
maxBackward: & max_maskedS = max(maskedS) &=> dmaskedS\_part2 => dmaskedS \\
 & maskedS = scaledS + mask \\
divBackward: & scaleedS = S/\sqrt{d} &=> dS \\
out\ softmax\\
bmmBackward: & &=>dQ, dK \\
\end{align}$$

#### (3)小结

- forward和backward中主要的memory space瓶颈在softmax；主要的memory load和store也发生在softmax。
- softmax的forward和backward中主要是element-wise操作，存在明显的memory bound。
- 结合算法内容看，存在三类优化机会：<font color=norange>**tiling, fusing, recomputation**</font>
  1. 以forward为例: s -> scaledS -> maskedS -> stableS -> expS 的连续运算方式，说明存在fuse机会，能直接减少memory load和store
  2. softmax中除了max和sum两个运算之外，都是element-wise operation。如果没有max和sum，则用tiling可以直接减少memory space cost。而'online softmax'论文提供了处理sum和max的方法，让tiling可行
  3. 在backward的计算中，需要用到forward pass中计算出来的expS或者P。如果只存sum(expS),则memory cost和load/store开销都能大幅降低。所以，在backward中用sum(expS)来recompute P可以进一步减少memory相关的两大cost——space和load/store。

### 3. 理论上的计算过程

$$\begin{align}
bmmBackward: dV & = P^TdA \\
dP & = dAV^T \\
softmaxbackward:temp &= P*dP \\
temp & = sum_{row}(temp)\\
temp &= dP - temp\\
dS & = P*(temp)\\
divBackward: dS & = dS/\sqrt{d} \\
dQ & = dSK \\
dK & = dS^TQ/\sqrt{d} \\
\end{align}$$

- 可见，直接用naive的方式用pytorch实现SDPA的计算过程并不高效。甚至不如直接按下面数学方式的memory高效。

## II. FA2的优化

### II.1 Forward pass

#### 1. 符号

   - 简化假设S只有1个row block，这个row block中含2个tiles。多个row block与一个的计算方式一样。$S=[S^{(1)}, S^{(2)}],其中S^{(1)}, S^{(2)}\in \mathbb{R} ^{B_r\times B_c}$
   - 简化假设V只有1个column block，这个column block中含2个tiles。$V=[V^{(1)}, V^{(2)}],其中V^{(1)}, V^{(2)}\in \mathbb{R} ^{B_c\times d}$
   - m表示max value in row block of S。
     - $m=max(rowmax(S^{(1)}), rowmax(S^{(2)}))\in \mathbb{R}^{B_r}$
   - l表示row sum of exp(stablized S)。
     - $l=rowsum(e^{S^{(1)}-m}) + rowsum(e^{S^{(2)}-m})\in \mathbb{R}^{B_r}$

#### 2. 示意图

   - 图中简化略去了max row的相关计算
<img src='pics/online_softmax.png' width='100%'>

#### 3. 计算思路

   - 目标是求$O=softmax(S)V=PV=P^{(1)}V^{(1)} + P^{(2)}V^{(2)}$
   - 由于计算$P^{(1)}$时用到的max和sum都需要第二组tiles中的信息，无法直接做tiling。解决方法是：
     1. 先只用第一组tiles计算出一个中间结果$\tilde{P}^{(1)}$
     2. 在计算第二组tiles的时候，用新的信息修正第一组结果
   - 尽可能用mm代替element-wise operation
   - 不存中间结果P，改存data更少但计算更麻烦(需要跨tile做修正)的sum of exp(S)，backward的时候做recomputation

#### 4. 优点：

1. 节省DRAM：硬件每次只处理一组tile，从而不需要大量的DRAM来存所有tiles使用的inputs,intermediate results和output的data
2. 减少data load/strore：operations fusing之后，intermediate result不需要存储。只有reduced data l需要存储，但它是reduced value，涉及的data量很小
3. 将element-wise div转mm来让高throughput的计算单元完成更多工作，提升计算效率
4. 通过recomputation进一步降低memory bound的程度

#### 5. 计算过程

- 先计算第一个tile
$$\begin{align}
m^{(1)} &= rowmax(S^{(1)}) \\
l^{(1)} & = rowsum(e^{S^{(1)}-m^{(1)}}) \\
\tilde{P} ^{(1)} & = e^{S^{(1)}-m^{(1)}}/l^{(1)} = diag(l^{(1)})^{-1}e^{S^{(1)}-m^{(1)}},element-wise转mm\\
\tilde{O}^{(1)} & = \tilde{P} ^{(1)}V^{(1)}\\
\end{align}$$

- 计算第二个tile的同时修正第一个tile的结果
$$\begin{align}
m^{(2)} &= rowmax(S^{(2)}) , m = max(m^{(1)}, m^{(2)}) \\
l^{(2)} & = rowsum(e^{S^{(2)}-m}) , l = e^{m^{(1)} - m}*l^{(1)} + l^{(2)} \\
\tilde{P} ^{(2)} & = diag(l)^{-1}e^{S^{(2)}-m},element-wise转mm\\
\tilde{O}^{(2)} & = \tilde{P} ^{(2)}V^{(2)}, O = diag\left(l^{(1)}/l\right)diag\left(e^{m^{(1)}-m}\right)\tilde{O}^{(1)} + \tilde{O}^{(2)}\\
\end{align}$$

- 可以使用的2个trick：
  1. 第一个tile中计算$\tilde{P} ^{(1)}$时使用的div操作在第二个tile时会被冲销掉，所以可以不做div，在第二个tile中也不冲销，这样就能减少运算。
     - 但分母的sub m的操作虽然在第二个tile也会重做，但不能同方法处理，因为那个操作关系到计算的stability。
  2. 理论上需要为backward存m和l来recompute P。recompute的方式是$P=e^{S-m}/\ell$。可以将其做以下转换，从而只用保存一组与m和l的shape相同的data，$L=m+log(\ell)$。这里要将$div(e^m*\ell)$转为log形式同样是因为stability。
  $$P=e^{S-m}/\ell = e^S/(e^m*\ell)=e^{S-log(e^m*\ell)}=e^{S-L}$$

- 加上trick后的计算方法：
$$\begin{align}
m^{(1)} &= \text{rowmax}\left(\mathbf{S}^{(1)}\right) \\
\tilde P^{(1)} & = e^{\mathbf{S}^{(1)} - m^{(1)}}\\
\ell^{(1)} &= \text{rowsum}\left(\tilde P^{(1)}\right) \\
\tilde{O}^{(1)} &= e^{\mathbf{S}^{(1)} - m^{(1)}} \mathbf{V}^{(1)} \\
\\
m^{(2)}& = \text{rowmax}\left(\mathbf{S}^{(2)}\right) ,m = \max\left(m^{(1)}, m^{(2)}\right) \\
\tilde P^{(2)} & = e^{\mathbf{S}^{(2)} - m}\\
\ell^{(2)} &= \text{rowsum}\left(\tilde P^{(2)}\right) , \ell = e^{m^{(1)} - m} \ell^{(1)} + \ell^{(2)} \\
\tilde{O}^{(2)} &= e^{\mathbf{S}^{(2)} - m} \mathbf{V}^{(2)}, \tilde{O} = \text{diag}(e^{m^{(1)} - m})^{-1} \tilde{O}^{(1)} + \tilde{O}^{(2)}\\
\\
O &= \text{diag}(\ell)^{-1} \tilde{O}
\end{align}$$

- 内循环遍历tile的计算过程一般化为：
$$\begin{align}
循环前初始化：m & =-\infty, \ell=0, \tilde{O} = 0\\
内循环遍历tiles：\\
m^{(j)}& = max\left(m^{(j-1)}, \text{rowmax}\left(\mathbf{S}^{(j)}\right)\right)\\
\tilde P^{(j)} & = e^{\mathbf{S}^{(j)} - m^{(j)}}\\
\ell^{(j)} &=e^{m^{(j-1)} - m^{(j)}} \ell^{(j-1)} +  \text{rowsum}\left(\tilde P^{(j)}\right)\\
\tilde{O}^{(j)} &= \text{diag}(e^{m^{(j-1)} - m^{(j)}})^{-1} \tilde{O}^{(j-1)} + \tilde P^{(j)} \mathbf{V}^{(j)}\\
内循环结束后：\\
O &= \text{diag}(\ell^{(j)})^{-1} \tilde{O}^{(j)}
\end{align}$$


#### 6. 算法

   - <img src='pics/fa2.png' width='80%'>

- **算法实现时选择的loop顺序很重要**：
  - **并行/串行方式**：做col做inner loop。这个顺序和FA1相反，带来了很大的improvement。
    - FA2的outer loop并行处理S的row blocks，即每个并行的kernel处理一个由Br个rows组成的row block。
    - 每个形状为(Br,d)的row blocks在具体计算时又分成更小的形为(Br,Bc)的tiles。在inner loop中每次iter处理一个tiles。
  - **与计算逻辑一致**：outer loop的对象是并行执行，inner loop的对象是顺序执行。根据前述计算过程，row block之间没有dependency，可以并行。row block内部各个tiles之间存在dependency，后序计算依赖前序计算的m应该顺序执行，因为前后tiles之间。所以FA2的顺序符合算法逻辑。
  - **尽可能少的内存读写开销**：DRAM load/write。
    - 当内循环是loop along tiles in a row block时，cross tiles计算m^(j)和l^(j)在同一个kernel内顺序进行，在得到最后的O,m和l之前不用存O^(j), m^(j)和l^(j)的中间值。
    - 但如果内循环是loop along tiles in a column block，则每个tile上计算出来的中间值O^(i), m^(i)和l^(i)都要存到DRAM。此时多出了所有中间值存取的开销。
  - <font color=lightblue>备注：假设原矩阵形状为(N,d)，row block是由多个rows组成的block，形为(Br,d),Br < N。column block是由多个columns组成的block，形为(N,Bc)，Bc < d。</font>

- **算法实现使用的block切分方式也很重要**：
  - FA1中，每个block处理一个1个attention head，也就是一共有batchsize * num_attention_head个block。也就是在batch和head两个维度上做并行。
    - 每个block会放入一个SM，每个SM中可以放多个blocks。A100为例，共108个SM。如果模型本身的block数量很大时，SM能充分activate，如果block数量少，那么硬件中会有SM闲置，导致occupancy很低。
    - 此外，当sequence length很大时，受DRAM限制，通常batchsize很很小，也会导致occupancy太低。
  - FA2中，进一步在sequence length这个dim上做并行。每个block只处理一个head中的部分rows。也就是将算法中的outer loop放到几个block来完成，而不是一个block完成整个outer loop。<font color=norange>**这样就能直接提高occupancy。**</font><font color=green>这一点也只能在outer loop在 loop along row时可以实现，如果是along column，则会因为column之间有dependency而无法实现。</font>

### II.2 Backward pass

#### 1. 计算过程

- math特征：
  - backward和forward不同的地方在于，唯一涉及reduce along row的操作可以用input matrix O和dO计算，不像在forward中那样，计算m和l是在中间结果S上完成。这里的好处是，可以先做reduce，然后再做loop。这样loop along row的时候row tiles之间没有dependency，可以并行，不像在forward中那样因为reduce只得到部分结果而产生row tiles之间的dependency。
$$dS = P*(dP - sum_{row}(P*dP)) = P*(dP - sum_{row}(O*dO))$$

- 梯度计算过程
$$\begin{align}
{\color{Orange} dV}  & = P^TdO \\
D & = sum_{row}(O*dO)\\
dP & = dOV^T \\
dS & = P*(dP - D)\\
{\color{Orange} dQ}  & = dSK/\sqrt{d} \\
{\color{Orange} dK}  & = dS^TQ/\sqrt{d} \\
\end{align}$$

- 第一个kernel：先计算$D=sum_{row}(O*dO)$
- 第二个kernel：外循环loop along column， 内循环loop along row
  - 按column切block的矩阵包括：K^T, V^T。实际上等价于按row切K和V。shape是(B_c, d), B_c指column block中包含的column数，d是d_model。
  - 按row切block的矩阵包括：Q, dO, L和D。Q和dO的shape是(B_r, d), L和D是(Br, )。B_r指row block中包含的row数。
  - for j in range(num_col_block): <font color=lightgreen>**[并行]**</font>
    - 从DRAM<font color=norange>**读取column block K^T_j和V^T_j**</font>，等价于读row block K_j和V_j
    - 初始化dK_j和dV_j
    - for i in range(num_row_block): <font color=lightgreen>**[串行]**</font>
      - <font color=norange>**读row block Q_i**</font>，计算$S_{ij}=Q_iK^T_j$。S_ij是一个shape为(B_r, B_c)的tile。
      - <font color=norange>**读L_i**</font>，计算$P_{ij} = exp(S_{ij}-L_i)$
      - <font color=norange>**读row block dO_i**</font>，计算$dP_{ij} = dO_iV^T_j$
      - <font color=norange>**读D_i**</font>，计算$dS_{ij} = P_{ij} * (dP_{ij}-D_i)$
      - 累积一次dV_i: $dV_i \leftarrow dV_i + P_{ij}^TdO_i$
      - <font color=red>**读dQ_i**</font>，累积$dQ_i \leftarrow dQ_i + dS_{ij}K_j$，<font color=red>**写dQ_i**</font>。K_j是jth row block of K。 <font color=lightgreen>[这一步发生重复读写，因为相同row block index i，不同column block index j对应的S_ij在cuda中是不同block of threads计算的，在triton中是不同program instance计算的。]</font>
      - 累积一次dK: $dK_i \leftarrow dK_i + dS_{ij}^TQ_i$
    - 将计算得到的dK_j和dV_j写入DRAM


- **外循环不变，内循环使用并行和串行策略的差异**：
  1. 并行：
     - 优点：由于D已经提前计算，每个tile of dS的计算全都可以并行。内循环并行可以让S的计算更快
     - 缺点：如果内循环并行，那么内循环遍历不同i值时计算出来的S_ij位于不同的block or PI上。此时$dK_j \leftarrow dK_j + dS_{ij}Q_i$就要做跨block or PI的累加。多个block or PI写同一个地址时会发生顺序读写瓶颈。同理，dV也一样。
     - 小结：此时dQ,dK,dV在内循环过程中每次iter都要从DRAM读写一次。

  2. 串行：
     - 缺点：原本可以并行的计算的tile of dS这时候只能在同一个row block上的不同column tiles上并行。跨row只能串行。所以tiles of S的计算成本会增加。
     - 优点：计算出来的P和dS中，同一个column block上不同的tiles都在同一个block or PI内。对应dS^T和P^T的row block上不同tiles在同一个block or PI内。此时，计算$dK=dS^TQ和 dV = P^TdO$时，就没有跨PI读写的需求。
     - 小结：此时dK,dV只在内循环结尾时写1次，dQ在内循环过程中每次iter都要从DRAM读写一次。


- **外循环改为遍历row，内存换遍历along columns的话**：
  - K,V在inner loop中每个iter上都要读入，dK，dV则每个iter都要读写；Q只在外循环中每个iter读一次，dQ只在整个inner loop结束时读写一次。和FA2中的方式相比，正好是K，V与Q的处理方式反过来。

#### 2. 算法

   - <img src='pics/fa2_bp.png' width='80%'>

- loop顺序和forward不同，这里outer loop是along columns，inner loop是along rows。

- block的划分方式如下：
   - <img src='pics/fa2_block.png' width='50%'>